<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/power_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PowerInfer: Fast Large Language Model Serving with a Consumer-grade GPU

PowerInfer is a CPU/GPU LLM inference engine leveraging activation locality for your device.



In [ ]:
!lscpu
!nvidia-smi

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.31
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [ ]:
#@title installation
!git clone https://github.com/SJTU-IPADS/PowerInfer
%cd /content/PowerInfer
!pip3 install -q -r requirements.txt

Cloning into 'PowerInfer'...
remote: Enumerating objects: 8876, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 8876 (delta 366), reused 524 (delta 263), pack-reused 8217
Receiving objects: 100% (8876/8876), 12.19 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (6180/6180), done.
/content/PowerInfer
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.8 MB/s eta 0:00:00


In [ ]:
#@title build binary
%cd /content/PowerInfer
import torch

if torch.cuda.is_available():
    print("build with GPU")
    !cmake -S . -B build -DLLAMA_CUBLAS=ON
    !cmake --build build --config Release
else:
    print("build with CPU")
    !cmake -S . -B build
    !cmake --build build --config Release

/content/PowerInfer
build with GPU
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Found CUDAToolkit: /usr/local/cuda/include (found version "12.2.140") 
-- cuBLAS found
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detect

In [ ]:
#@title download Llama3-8B-Chinese-Chat-GGUF-4bit weights
%cd /content/PowerInfer
!mkdir -p /content/PowerInfer/Llama3-8B-Chinese-Chat-GGUF-4bit

!cd Llama3-8B-Chinese-Chat-GGUF-4bit && wget https://huggingface.co/shenzhi-wang/Llama3-8B-Chinese-Chat-GGUF-4bit/resolve/main/config.json
!cd Llama3-8B-Chinese-Chat-GGUF-4bit && wget https://huggingface.co/shenzhi-wang/Llama3-8B-Chinese-Chat-GGUF-4bit/resolve/main/Llama3-8B-Chinese-Chat-q4_0-v2_1.gguf


/content/PowerInfer
--2024-06-17 03:27:28--  https://huggingface.co/shenzhi-wang/Llama3-8B-Chinese-Chat-GGUF-4bit/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 13.33.30.114, 13.33.30.23, 13.33.30.76, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 649 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     649  --.-KB/s    in 0s      

2024-06-17 03:27:29 (348 MB/s) - ‘config.json’ saved [649/649]

--2024-06-17 03:27:29--  https://huggingface.co/shenzhi-wang/Llama3-8B-Chinese-Chat-GGUF-4bit/resolve/main/Llama3-8B-Chinese-Chat-q4_0-v2_1.gguf
Resolving huggingface.co (huggingface.co)... 13.33.30.114, 13.33.30.23, 13.33.30.76, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b8/19/b8190a9fdff5739e6064ef01f1c841c

In [ ]:
#@title inference
%cd /content/PowerInfer
PROMPT="你好啊，请给我讲个简短的笑话吧" #111@param {type:"string"}
PROMPT += "\n"
import datetime
print(datetime.datetime.now())
# https://huggingface.co/shenzhi-wang/Llama3-70B-Chinese-Chat-GGUF-4bit

import torch

if torch.cuda.is_available():
    print("run with GPU")
    !./build/bin/main -m ./Llama3-8B-Chinese-Chat-GGUF-4bit/Llama3-8B-Chinese-Chat-q4_0-v2_1.gguf -n 128 -t 8 -p {PROMPT} --vram-budget 15
else:
    print("run with CPU")
    !./build/bin/main -m ./Llama3-8B-Chinese-Chat-GGUF-4bit/Llama3-8B-Chinese-Chat-q4_0-v2_1.gguf -n 128 -t 8 -p {PROMPT}


print(datetime.datetime.now())

/content/PowerInfer
2024-06-17 03:31:06.590677
using GPU
Log start
main: build = 1581 (4896dfb)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1718595066
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ./Llama3-8B-Chinese-Chat-GGUF-4bit/Llama3-8B-Chinese-Chat-q4_0-v2_1.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 128256,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  4096, 14336,     1,     1 ]
llama_model